In [43]:
# Imports
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import sys
os.path.dirname(sys.executable)
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.edge.service import Service
from selenium.webdriver.safari.service import Service

import pytest
import time

In [44]:
### Importing EINs

sheet_id = '1vKhl-3EW_UL2fHpCHtsynu6S9T31REsWptKA2uY2NV0'
sheet_name = 'ENVS_Animal'
url = 'https://docs.google.com/spreadsheets/d/'+ sheet_id + '/gviz/tq?tqx=out:csv&sheet=' + sheet_name

df = pd.read_csv(url, header=0)

envs_groups = {}
for row in df.iterrows():
    group = row[1]["recipient_name"]
    ein = row[1]["EIN"]
    envs_groups[group] = ein


In [45]:
### Scraping from Guidestar

guidestar_url = 'https://www.guidestar.org/Account/Login?returnUrl=https%3A%2F%2Fwww.guidestar.org%2F'

# driver = webdriver.Chrome(executable_path='/Users/finnianlowden/opt/anaconda3/chromedriver')
driver = webdriver.Chrome(executable_path='/Users/finn/opt/anaconda3/chromedriver')
driver.get(guidestar_url)

# Logging into Guidestar
driver.find_element_by_id('emailAddress').send_keys('galen_hall@brown.edu')
driver.find_element_by_id('password').send_keys('MiNaReT2020!')
driver.find_element_by_id('signIn').click()


<ipython-input-45-7f42d5dc3179>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/finn/opt/anaconda3/chromedriver')
<ipython-input-45-7f42d5dc3179>:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('emailAddress').send_keys('galen_hall@brown.edu')
<ipython-input-45-7f42d5dc3179>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('password').send_keys('MiNaReT2020!')
<ipython-input-45-7f42d5dc3179>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('signIn').click()


In [46]:
# Going to each ENVS group's page
group_EINs = envs_groups
group_financials = {}
for group in group_EINs:
    
    ein = envs_groups[group]
    driver.find_element_by_id('Keywords').clear()
    driver.find_element_by_id('Keywords').send_keys(ein)
    driver.find_element_by_id('searchBarBtn').click()
    
    time.sleep(5)
    
    page_source = driver.page_source
    HTML_tree = BeautifulSoup(page_source)
    
    revenueType = HTML_tree.find_all("p", class_="fin-value")
    
    count = 0
    financials = {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
    
    for revenue in revenueType:
        text = str(revenue)
        clean_rev = text.replace('<p class="fin-value"> ', '')
        clean_rev = clean_rev.replace(' </p>', '')
        clean_rev = clean_rev.replace('$', '')
        clean_rev = clean_rev.replace(',', '')
        if count == 0:
            financials['TOTAL REVENUE'] = clean_rev
        if count == 1:
            financials['GROSS RECEIPTS'] = clean_rev
        else:
            financials['ASSETS'] = clean_rev
        count += 1
    group_financials[group] = financials
    
    print(group, group_financials[group])
    
    driver.back()


<ipython-input-46-d05f4e8704ab>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('Keywords').clear()
<ipython-input-46-d05f4e8704ab>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('Keywords').send_keys(ein)
<ipython-input-46-d05f4e8704ab>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('searchBarBtn').click()


NATIONAL FISH AND WILDLIFE FOUNDATION {'TOTAL REVENUE': '287239368', 'GROSS RECEIPTS': '500199617', 'ASSETS': '2847334651'}
CALIFORNIA FOUNDATION ON THE ENVIRONMENT AND THE ECONOMY {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': 'n/a'}
NATURE CONSERVANCY {'TOTAL REVENUE': '1137390674', 'GROSS RECEIPTS': '0', 'ASSETS': '7975541090'}
STUDENT CONSERVATION ASSOCIATION {'TOTAL REVENUE': '37357144', 'GROSS RECEIPTS': '38526776', 'ASSETS': '20770923'}
CONSERVATION INTERNATIONAL {'TOTAL REVENUE': '159498081', 'GROSS RECEIPTS': '289337802', 'ASSETS': '403472995'}
AMERICAN FORESTS {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': 'n/a'}
NATIONAL PARK FOUNDATION/SERVICE {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': 'n/a'}
RESOURCES FOR THE FUTURE {'TOTAL REVENUE': '9153096', 'GROSS RECEIPTS': '17679092', 'ASSETS': '83427208'}
ANNAPOLIS CENTER FOR SCIENCE-BASED PUBLIC POLICY {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': 'n/a'}
WORLD WILDLIFE FUND {'TOTAL R

NEW ENGLAND FORESTRY {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
ALASKA SEALIFE CENTER {'TOTAL REVENUE': '7806179', 'GROSS RECEIPTS': '9545337', 'ASSETS': '40459525'}
TEXAS PARKS AND WILDLIFE {'TOTAL REVENUE': '9315562', 'GROSS RECEIPTS': '17498904', 'ASSETS': '64035045'}
INFORM {'TOTAL REVENUE': '17310', 'GROSS RECEIPTS': '3876', 'ASSETS': '17310'}
TROUT UNLIMITED {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
AMERICAN FISHERIES SOCIETY {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
SOLAR ELECTRIC LIGHT {'TOTAL REVENUE': '2173289', 'GROSS RECEIPTS': '1683373', 'ASSETS': '955150'}
LOUISIANA RESOURCE CENTER FOR EDUCATORS {'TOTAL REVENUE': '1496996', 'GROSS RECEIPTS': '1496996', 'ASSETS': '3391880'}
GREATER CARIBBEAN ENERGY AND ENVIRONMENT {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
BAYOU LAND CONSERVANCY {'TOTAL REVENUE': '364970', 'GROSS RECEIPTS': '2943672', 'ASSETS': '2012174'}
TEXAS CONSERVATION {'TOTAL REVENUE': '63223', 'GROSS RECEIPTS

GREEN PROJECT {'TOTAL REVENUE': '551296', 'GROSS RECEIPTS': '567460', 'ASSETS': '567220'}
ADOPT A CAT {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
AMERICAN EAGLE {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
DAUPHIN ISLAND SEA LAB {'TOTAL REVENUE': '307881', 'GROSS RECEIPTS': '329386', 'ASSETS': '2640971'}
VIRGINIA ZOOLOGICAL SOCIETY {'TOTAL REVENUE': '2531232', 'GROSS RECEIPTS': '3929202', 'ASSETS': '4647967'}
FRIENDS OF THE EDDIE V GRAY WETLANDS EDUCATION AND RESEARCH CENTER {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': 'n/a'}
ABANDONED ANIMAL RESCUE {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
FREEDOM STREET RESCUE {'TOTAL REVENUE': '182228', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '182228'}
ZOOLOGICAL SOCIETY OF PHILADELPHIA {'TOTAL REVENUE': '41038826', 'GROSS RECEIPTS': '43427774', 'ASSETS': '113302279'}
GREEN BUILDING INITIATIVE {'TOTAL REVENUE': '2720995', 'GROSS RECEIPTS': '4944362', 'ASSETS': '5229305'}
RIVER OAKS BAPTIST SC

BRONX RIVER RESTORATION PROJECT {'TOTAL REVENUE': '6', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '6'}
NATIONAL AQUARIUM {'TOTAL REVENUE': '50248514', 'GROSS RECEIPTS': '25536884', 'ASSETS': '104725819'}
EAST COAST GREENWAY ALLIANCE {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
LARAMIE RAPTOR REFUGE {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
LIVING DESERT WILDLIFE AND BOTANICAL PARK {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': 'n/a'}
NATIONAL WILDLIFE REHABILITATORS ASSOCIATION {'TOTAL REVENUE': '322623', 'GROSS RECEIPTS': '397251', 'ASSETS': '1003031'}
OPENLANDS {'TOTAL REVENUE': '7148714', 'GROSS RECEIPTS': '12439922', 'ASSETS': '33720529'}
OUR LAND SOCIETY {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
ALTERNATIVE ENERGY RESOURCESANIZATION {'TOTAL REVENUE': '388095', 'GROSS RECEIPTS': '388095', 'ASSETS': '237581'}
SENECA PARK ZOO SOCIETY {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': 'n/a'}
COUNCIL ON THE ENVIRONMENT {'TOT

ALASKA SONGBIRD INSTITUTE {'TOTAL REVENUE': '136072', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '136072'}
MONTANA AQUATIC RESOURCES SERVICES {'TOTAL REVENUE': '566593', 'GROSS RECEIPTS': '566593', 'ASSETS': '2402880'}
PANDRILLUS USA {'TOTAL REVENUE': '165801', 'GROSS RECEIPTS': '165801', 'ASSETS': '68520'}
ANIMAL DEFENSE LEAGUE {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
SCOUTS HONOR RESCUE {'TOTAL REVENUE': '219927', 'GROSS RECEIPTS': '111395', 'ASSETS': '72396'}
BAY NATURE INSTITUTE {'TOTAL REVENUE': '882546', 'GROSS RECEIPTS': '987401', 'ASSETS': '312426'}
CONEG POLICY RESEARCH CENTER {'TOTAL REVENUE': '542482', 'GROSS RECEIPTS': '548198', 'ASSETS': '1453909'}
HANDS ON - A CHILDS GALLERY {'TOTAL REVENUE': '488445', 'GROSS RECEIPTS': '622625', 'ASSETS': '828651'}
FREEPORT RENAISSANCE ASSOCIATION {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
MISSISSIPPI RIVER CITIES AND TOWNS INITIATIVE {'TOTAL REVENUE': '708262', 'GROSS RECEIPTS': '557195', 'ASSETS': '1010273'}
T

FARM SANCTUARY {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
PAWS CHICAGO {'TOTAL REVENUE': '18707575', 'GROSS RECEIPTS': '45895338', 'ASSETS': '76633109'}
NATIONAL ARBOR DAY {'TOTAL REVENUE': '83851981', 'GROSS RECEIPTS': '86236100', 'ASSETS': '72817945'}
WESTERN PENNSYLVANIA CONSERVANCY {'TOTAL REVENUE': '24653253', 'GROSS RECEIPTS': '33059965', 'ASSETS': '105381884'}
ANIMAL LEGAL DEFENSE {'TOTAL REVENUE': '13913508', 'GROSS RECEIPTS': '14432483', 'ASSETS': '15929701'}
AUSBON SARGENT LAND PRESERVATION TRUST {'TOTAL REVENUE': '2765843', 'GROSS RECEIPTS': '4364429', 'ASSETS': '6062840'}
HOUSTON BOXER RESCUE {'TOTAL REVENUE': '103101', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '103101'}
RITA B HUFF HUMANE SOCIETY OF WALKER COUNTY {'TOTAL REVENUE': '800416', 'GROSS RECEIPTS': '800416', 'ASSETS': '1539246'}
SOARING EAGLE {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
TANGLEWOOD {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
IDAHO RIVERS UNITED {'TOTAL REV

CHESAPEAKE BAY {'TOTAL REVENUE': '37635927', 'GROSS RECEIPTS': '48544018', 'ASSETS': '126294841'}
GRASS RIVER NATURAL AREA {'TOTAL REVENUE': '576890', 'GROSS RECEIPTS': '495653', 'ASSETS': '666339'}
MIDDLEBURG HUMANE {'TOTAL REVENUE': '2207750', 'GROSS RECEIPTS': '1334484', 'ASSETS': '6113641'}
CLEARWATER MARINE AQUARIUM {'TOTAL REVENUE': '982835', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '982835'}
WESTERN PENNSYLVANIA HUMANE SOCIETY {'TOTAL REVENUE': '667540', 'GROSS RECEIPTS': '922297', 'ASSETS': '0'}
HELEN WOODWARD ANIMAL CENTER {'TOTAL REVENUE': '14519271', 'GROSS RECEIPTS': '15315857', 'ASSETS': '40048277'}
LAKES REGION CONSERVATION TRUST {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
CLEAN WATER {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
HAVEN HUMANE SOCIETY {'TOTAL REVENUE': '3888706', 'GROSS RECEIPTS': '3935508', 'ASSETS': '13626470'}
RAINFOREST ALLIANCE {'TOTAL REVENUE': '46704876', 'GROSS RECEIPTS': '47852037', 'ASSETS': '34755185'}
ALAQUA ANIMAL REFUGE 

PAWS PLACE {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
RESCUE DOGS ROCK {'TOTAL REVENUE': '3426207', 'GROSS RECEIPTS': '3426207', 'ASSETS': '1281878'}
TRACYS DOGS {'TOTAL REVENUE': '617037', 'GROSS RECEIPTS': '617037', 'ASSETS': '611963'}
WILDLIFE ALLIANCE {'TOTAL REVENUE': '6519692', 'GROSS RECEIPTS': '6935934', 'ASSETS': '4827091'}
EAST MISSISSIPPI ANIMAL RESCUE {'TOTAL REVENUE': '58584', 'GROSS RECEIPTS': '58584', 'ASSETS': '64390'}
TRUSTEES OF RESERVATIONS {'TOTAL REVENUE': 0, 'GROSS RECEIPTS': 0, 'ASSETS': 0}
SAN FRANCISCO BAY BIRD OBSERVATORY {'TOTAL REVENUE': '1037549', 'GROSS RECEIPTS': '896261', 'ASSETS': '338329'}
ANGELS AMONGUNITED STATESPET RESCUE {'TOTAL REVENUE': '2408526', 'GROSS RECEIPTS': '2403082', 'ASSETS': '1538449'}
MIRACLE FARM {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
CABINET RESOURCE GROUP {'TOTAL REVENUE': '0', 'GROSS RECEIPTS': 'n/a', 'ASSETS': '0'}
BETTER FUTURE PROJECT {'TOTAL REVENUE': '870647', 'GROSS RECEIPTS': '161